In [1]:
# default_exp models

# models

> API details.

In [2]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [137]:
# export
import json
import bcrypt

In [4]:
# Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client_id, secret = open("../client.secret").read().strip().split(",")

In [17]:
# export
from strava_overview.imports import *
from strava_overview.s3 import *
from strava_overview.auth import *

In [25]:
# export
from pynamodb.models import Model
from pynamodb.attributes import (
    UnicodeAttribute, NumberAttribute, UnicodeSetAttribute, UTCDateTimeAttribute, BooleanAttribute, MapAttribute, ListAttribute,
)

True

In [132]:
# export
class EmailIndex(Model):
    """
    This class represents a global secondary index
    """
    class Meta:
        table_name = "motionreview-email"
        region = 'eu-central-1'
        read_capacity_units = 1
        write_capacity_units = 1
    # This attribute is the hash key for the index
    # Note that this attribute must also exist
    # in the model
    email = UnicodeAttribute(hash_key=True)
    id = NumberAttribute()
    password = UnicodeAttribute()
    lock = BooleanAttribute(default = False)
    
    @classmethod
    def get_(self, email:str, hard =False):
        emls = list(EmailIndex.query(email))
        if len(emls) == 0:
            raise KeyError(email)
        elif len(emls) == 1:
            return emls[0]
        else:
            if hard:
                return emls
            raise KeyError(email)
        
    @classmethod
    def exists(self, email:str):
        try:
            o = self.get_(email)
            if o is not None:
                return True
        except KeyError:
            return False
    

In [133]:
email = EmailIndex('test', id = 126, password = 'foobar')
email.save()
test_eq(str(EmailIndex.get('test')), str(email))
test_eq(EmailIndex.exists('foobar'), False)

In [134]:
# export
class AlreadyFoundExeption(Exception):pass
class IncorrectPassword(Exception):pass
class LockedAccount(Exception):pass

In [140]:
# export

types = [
            "time",
            "distance",
            "latlng",
            "altitude",
            "velocity_smooth",
            "moving",
            "grade_smooth",
            "temp",
            "watts",
            "cadence",
            "heartrate",
    ]

class Athlete(Model):
    class Meta:
        table_name = "motionreview-athlete"
        region = 'eu-central-1'
    id = NumberAttribute(hash_key=True)
    firstname = UnicodeAttribute()
    lastname = UnicodeAttribute()
    profile = UnicodeAttribute()
    sex = UnicodeAttribute()
    max_heartrate = NumberAttribute(null=True)
    ftp = NumberAttribute(null=True)
    weight = NumberAttribute(null=True)
    zones = ListAttribute(null=True)
    email = UnicodeAttribute(null=True)
    access_token = UnicodeAttribute(null=True)
    refresh_token = UnicodeAttribute(null=True)
    
    @classmethod
    def from_stravalib(cls, athlete:stravalib.model.Athlete, **kwargs):  
        dct = {k:athlete.__getattribute__(k) for k in 'firstname,lastname,profile,sex,max_heartrate,weight,ftp'.split(',') if (athlete.__getattribute__(k) is not None)}
        dct.update(kwargs)
        ath = cls(athlete.id,**dct)
        ath.save()
        return ath
        
    @classmethod
    def authenticate(cls, sleep_time = 60):
        client = stravalib.client.Client()
        go_strava_auth(client)
        tokens = Tokens()
        i = 0
        while i < sleep_time:
            i+=1
            time.sleep(1)
            id = tokens.is_new()
            if id:
                return cls.get_athlete(id = id)
        raise FileNotFoundError
        
    @classmethod
    def from_id(cls, id =None, **kwargs):
        if id is None:
            return cls.authenticate(**kwargs)
        return Athlete.get_athlete(id = id)
    
    @staticmethod
    def get_athlete(id = None, ath:stravalib.model.Athlete = None):
        if id is None:
            assert type(ath) == stravalib.model.Athlete
            id = ath.id

        aths =list(Athlete.query(int(id)))
        if len(aths) == 1:
            return aths[0]
        elif len(aths) == 0 and ath is not None:
            return Athlete.from_stravalib(ath)
        else:
            raise IndexError(f"to many athlets with id {ath.id}:{aths}")
            
    @staticmethod
    def login(email:str, password:str, trys:int):
        emls = EmailIndex.get_(email)
        if emls.lock:
            raise LockedAccount()
        master_secret_key, salt = os.getenv("mastersecret"),os.getenv("SALT")
        print(f"attemted {trys}/10 for {emls.email}")
        combo_password = password + salt + master_secret_key
        if bcrypt.checkpw(combo_password.encode(), emls.password.encode()):
            return Athlete.from_id(emls.id)
        else:
            if trys > 10:
                emls.lock = True
                raise LockedAccount()
            raise IncorrectPassword()   
            
    def add_login(self, email, password, hard = False):
        if EmailIndex.exists(email) and not hard:
            raise AlreadyFoundExeption()
        self.email = email
        self.save()
        master_secret_key, salt = os.getenv("mastersecret"),os.getenv("SALT")
        combo_password = password + salt + master_secret_key
        hashed_password = bcrypt.hashpw(combo_password.encode(), salt.encode())
        e = EmailIndex(email, id = self.id, password = hashed_password.decode())
        e.save()
        

In [141]:
ath = Athlete.get_athlete('5058582')

In [142]:
ath.add_login('takotabak@gmail.com','foobar', hard = True)

In [144]:
ath2 = Athlete.login('takotabak@gmail.com','foobar',0)
test_eq(str(ath2),str(ath))

attemted 0/10 for takotabak@gmail.com


In [145]:
Athlete.authenticate()

motionreview-athlete<5058582>

In [146]:
# export

types = [
            "time",
            "distance",
            "latlng",
            "altitude",
            "velocity_smooth",
            "moving",
            "grade_smooth",
            "temp",
            "watts",
            "cadence",
            "heartrate",
    ]

class Activity(Model):
    class Meta:
        table_name = "motionreview-activity"
        region = 'eu-central-1'
        
    id = UnicodeAttribute(hash_key=True)
    athlete_id = UnicodeAttribute()
    start_date_local = UTCDateTimeAttribute() 
    name = UnicodeAttribute() 
    act_type = UnicodeAttribute()
    device_watts = BooleanAttribute(default=False)  
    has_heartrate = BooleanAttribute(default=False)
    on_s3 = BooleanAttribute(default=False)
    ti = NumberAttribute(null=True)
    ti_w = NumberAttribute(null=True)
    ti_hr = NumberAttribute(null=True)
    
    @classmethod
    def from_stravalib(cls, act):
        Athlete.get_athlete(ath = act.athlete)        
        device = act.device_watts if act.device_watts else False
        act = cls(id = str(act.id), name = act.name, athlete_id = str(act.athlete.id), act_type = act.type, 
                        device_watts=device, start_date_local = act.start_date_local, has_heartrate = act.has_heartrate,
                        )     
        return act
    
    @classmethod
    def from_id(cls, id):
        return get_activity(id = id)
                
    @staticmethod
    def get_activity(id = None, act:stravalib.model.Activity = None):
        if id is None:
            assert type(act) == stravalib.model.Activity
            id = act.id
            
        acts =list(Activity.query(int(id)))
        if len(acts) == 1:
            return aths[0]
        elif len(acts) == 0:
            return Activity.from_stravalib(ath)
        else:
            raise IndexError(f"to many activitys with id {act.id}:{acts}")
            
    def save_stream(self, h):
        if not self.on_s3:
            streams = h.client.get_activity_streams(self.id, types=types, series_type="time")
            self.download_save(streams)
            
    def filename(self):        
        if not os.path.isdir(self.athlete_id):
            os.mkdir(self.athlete_id)
        return self.athlete_id +'/'+ self.id + '.csv'
    
    def download_save(self, streams):  
        df = pd.DataFrame()
        f = self.filename()
        # Write each row to a dataframe
        for item in types:
            if item in list(streams.keys()):
                df[item] = pd.Series(streams[item].data, index=None)
            df["act_id"] = self.id
            df["act_name"] = self.name
        df.to_csv(f, index=False)
        upload_file(f)
        self.on_s3 = True
        self.save()
        
    def get_df(self) -> pd.DataFrame:
        f = self.filename()
        download_file(f)
        return pd.read_csv(f)

In [147]:
# export
class Handel:
    def __init__(self, ath_id=None):
        self.client = stravalib.client.Client()
        self.athlete = Athlete.get_athlete(ath_id)
        self.set_access_token()        
        self.strava_athlete = self.client.get_athlete()
        
    def set_access_token(self):
        self.client.access_token = self.athlete.access_token
        try:            
            self.strava_athlete = self.client.get_athlete()
            return
        except:
            self.client.access_token = self.client.refresh_access_token(
                client_id=client_id,
                client_secret=secret,
                refresh_token=self.athlete.refresh_token,
            )     
        self.strava_athlete = self.client.get_athlete()       
            

In [148]:
h = Handel('5058582')
f"Welcome {h.athlete.firstname} {h.athlete.lastname}"

'Welcome Tako Tabak'

In [149]:
h.athlete.id

5058582

In [150]:
# export
def datetime_fix(start_date, days):
    before = pd.Timestamp(start_date)
    before = pd.Timestamp(before.year,before.month,before.day,23,59)
    before.tz_localize('utc')
    after = before - pd.Timedelta(days=days)
    return before, after

In [151]:
test_eq(datetime_fix(pd.Timestamp('2020'), 7),(pd.Timestamp('2020-01-01 23:59:00'), pd.Timestamp('2019-12-25 23:59:00'))
       )
test_eq(datetime_fix(pd.Timestamp('2020-03-1'), 7),(pd.Timestamp('2020-03-1 23:59:00'), pd.Timestamp('2020-02-23 23:59:00'))
       )

In [152]:
# export
def get_activities(self, start_date = None, days = 7):
    start_date = ifnone(start_date, pd.Timestamp.now())
    before, after = datetime_fix(start_date, days)
    # Returns a list of Strava activity objects, up to the number specified by limit
    activities = self.client.get_activities(before = before, after=after)
    return [Activity.from_stravalib(item) for item in activities]

Handel.get_activities = get_activities


In [153]:
h = Handel()
acts = h.get_activities()

AssertionError: 

In [154]:
acts[0].save()
acts[0].save_stream(h)

NameError: name 'acts' is not defined

In [155]:
# export
def save_activities(self, activities:[]):
    for i, act in enumerate(activities):
        act.save_stream(self)
    return activities
Handel.save_activities = save_activities

In [156]:
h = Handel()
acts = h.get_activities()
acts = h.save_activities(acts)

AssertionError: 

In [157]:
print(acts[0])

NameError: name 'acts' is not defined

In [158]:
df = acts[0].get_df()
test_eq(type(df),pd.DataFrame)
df.head()

NameError: name 'acts' is not defined

In [159]:
from nbdev.export import *
notebook2script()


Converted 00_s3.ipynb.
Converted 01_models.ipynb.
Converted 02_ti.ipynb.
Converted 04_auth.ipynb.
Converted 05_backend.ipynb.
Converted index.ipynb.
